Hate Speech Filtering

Using Tf-Idf and BoW to build SVM model

Linear model? Or RBF model?

RACIST texts classified with '1'
NON-RACIST texts classified with '0'


In [1]:
# Libraries

# Web scraper libraries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import sys
import csv

# ML libraries
import re 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import SVC
import matplotlib.pyplot as plt

# NLTK Stemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Logistic Regression
lreg = LogisticRegression()



In [2]:
def get_page_links(pageDriver, baseUrl):
    a_tags = driver.find_elements_by_tag_name('a')

    # Get all urls
    urls = [tag.get_attribute('href') for tag in a_tags]

    #Get urls that start with baseUrl
    useful_urls = [url for url in urls if url and baseUrl in url]

    return useful_urls

def get_page_text(pageDriver):
    pageBody = pageDriver.find_element_by_xpath("/html/body")
    if not pageBody:
        return ""
    return pageBody.text


visited_urls = set()
unvisited_urls = set()
error_urls = set()

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--log-level=1')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

baseUrl = input("Please enter url: ")
unvisited_urls.add(baseUrl)

ct = 0
while unvisited_urls:
    # Remove url from unvisited and add it to visited
    currPage = unvisited_urls.pop()
    print(currPage)
    visited_urls.add(currPage)

    # Get page content
    try:
        driver.get(currPage)
    except:
        error_urls.add(currPage)
        print('Error getting page: ', currPage)
        continue

    # Get links
    try:
        currLinks = get_page_links(driver, baseUrl)
    except:
        error_urls.add(currPage)
        print('Error getting links: ', currPage)
        continue

    # Get text
    try:
        currText = get_page_text(driver)
        textToWrite = currText.splitlines()
        with open(f'testCSV.csv', 'w') as csvfile:
            fieldnames = ['id', 'text']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for text in textToWrite:
                writer.writerow({'id': ct, 'text': {text}})
                ct += 1
    except:
        error_urls.add(currPage)
        print('Error getting text: ', currPage)
        continue

print("WEB SCRAPING FINISHED")



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64_m1.zip
Driver has been saved in cache [/Users/tonycui/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60]


https://www.splcenter.org/fighting-hate/extremist-files/group/stormfront
WEB SCRAPING FINISHED


In [3]:
# Read data pandas, keep raw copy of data
train_RAW = pd.read_csv('train.csv')
test_RAW = pd.read_csv('testCSV.csv')
train = train_RAW
test = test_RAW


TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.
This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

Ref: 
    https://monkeylearn.com/blog/what-is-tf-idf/#:~:text=TF%2DIDF%20(term%20frequency%2D,across%20a%20set%20of%20documents


Doc:
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [4]:
# TF-IDF freq
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Bag of words is a Natural Language Processing technique of text modelling. 
In technical terms, we can say that it is a method of feature extraction with text data.

Ref: 
    https://www.mygreatlearning.com/blog/bag-of-words/


Doc:
    https://scikit-learn.org/stable/modules/feature_extraction.html

In [5]:
# Bag Of Words Model
bow_vectorizer = CountVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

Ref/Doc:
    https://scikit-learn.org/stable/modules/svm.html#:~:text=Support%20vector%20machines%20(SVMs)%20are,than%20the%20number%20of%20samples

In [6]:
# Combine training data with test data
allTexts = train.append(test, ignore_index=True, sort=False)

allTexts.tail()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_61670/4241641124.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allTexts = train.append(test, ignore_index=True, sort=False)


,id,label,text
32034,72,NaN,{'Hatewatch'}
32035,73,NaN,{'Intelligence Report'}
32036,74,NaN,{'Publications'}
32037,75,NaN,{'Law Enforcement'}
32038,76,NaN,{'Activist Toolkits'}


In [8]:
def clean_texts(text, pattern):
    process = re.findall(pattern, text)
    for i in process:
        text = re.sub(i, '', text) 
    return text

In [ ]:
# Clean text 
# Add new column 'tidy_tweet'

# Clean user handles (@user_name)
allTexts['tidy_text'] = np.vectorize(clean_texts)(
    allTexts['text'], "@[\w]*") 

# Clean non-alphabets
allTexts['tidy_text'] = allTexts['tidy_text'].str.replace(
    "[^a-zA-Z]", " ") 

allTexts.tail()

In [ ]:
# Tokenization
tokenized_text = allTexts['tidy_text'].apply(lambda x: x.split())

tokenized_text.tail()

In [ ]:
tokenized_text = tokenized_text.apply(
    lambda x: [lemmatizer.lemmatize(i) for i in x])  

# Combine tokens back together
for i in range(len(tokenized_text)):
    tokenized_text[i] = ' '.join(tokenized_text[i])

# Replace with lemmatized
allTexts['tidy_text'] = tokenized_text

allTexts.tail()


In [ ]:
# Feature extraction - Bag-of-Words [sklearn’s CountVectorizer]
# ------------------------------------------------------------------------------------------------------------------

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(allTexts['tidy_text'])

print(bow)

In [ ]:

# TF-IDF Features - Looks at frequency of occurence for terms/importance of the term

# TF = (Number of times term t appears in a document)/(Number of terms in the document)
# IDF = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
# 
# TF-IDF = TF*IDF

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(allTexts['tidy_text'])

print(tfidf)


In [ ]:
train_bow = bow[:31962, :]
test_bow = bow[31962:, :]

# splitting data into training and validation set



# xtrain_bow training dataset
# xvalid_bow validation for training
# ytrain label vector
# yvalid validation label vector
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(
    train_bow, train['label'], test_size=0.1)

# train_bow feature matrix
# train['label'] label vector
# random_state shuffles data before split into training and testing
# test_size percentage of data gets tested on (0.9 training)


# print(xtrain_bow)
# print(xvalid_bow)
print(ytrain)
# print(yvalid)

# lreg = LogisticRegression()
# lreg.fit(xtrain_bow, ytrain)    
# train using training dataset and label vector

In [ ]:
# tfidf feature matrix
train_tfidf = tfidf[:31962, :]
test_tfidf = tfidf[31962:, :]


# ytrain.index index of axis labels 
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

# extract label vectors from the feature matrix via matching index

# print(xtrain_tfidf)


# fit on logistic regression
# lreg.fit(xtrain_tfidf, ytrain) 

In [ ]:
# SVM
svm = SVC(kernel='rbf')

SVM tutorial: https://www.youtube.com/watch?v=FB5EdxAGxQg

In [ ]:
svm.fit(xtrain_bow, ytrain)        # Build using bow

lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain)                     # training the model

# predicting on the validation set
prediction = lreg.predict_proba(xvalid_bow)
# if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction[:, 1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)  # calculating f1 score



# accuracy test
# svm.score(xvalid_bow, yvalid)       

In [ ]:
test_pred = lreg.predict_proba(test_bow)
test_pred_int = test_pred[:, 1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)

# If any texts are classified as '1'
# Then the website contains hate speech to some extent
if 1 in test_pred_int:
    print("It is racist")
test['label'] = test_pred_int
submission = test[['id', 'label']]
# writing data to a CSV file
submission.to_csv('predict_lreg_bow.csv', index=False)

In [ ]:
svm.fit(xtrain_tfidf, ytrain)      # Build using TD-IDF


svm.score(xvalid_tfidf, yvalid)


In [ ]:
svm.predict(test_tfidf)

# if '1' in svm.predict(test_tfidf):
#     print("it is racist")

In [ ]:
svm.predict(test_bow)

In [ ]:
prediction = svm.predict(xvalid_bow)
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int)